The following cells are for: 
- Importing the relevant datasets and packages
- Manipulating and joining individual datasets into aggregate tables for Exploratory Analysis and Modelling 

In [1]:
# Importing Relevant Packages
import numpy as np
import pandas as pd

In [2]:
# Importing YouTube Datasets
video_description_df = pd.read_csv("/home/chigoz/Downloads/Disso/00. Inputs/Video_Description_Table.csv", delimiter = ";")
video_statistics_df = pd.read_csv("/home/chigoz/Downloads/Disso/00. Inputs/Video_Statistics_Table.csv", delimiter = ";")
video_comments_df = pd.read_csv("/home/chigoz/Downloads/Disso/00. Inputs/Video_Comments_Table.csv", delimiter = ";")
video_tags_df = pd.read_csv("/home/chigoz/Downloads/Disso/00. Inputs/Video_Tags_Table.csv", delimiter = ";")

# Importing Google Analytics Dataset
ga_df = pd.read_csv("/home/chigoz/Downloads/Disso/00. Inputs/Google_Analytics_Users.csv")

## Google Analytics Data

To be used for Exploratory Analysis.

In [3]:
# Converting the date column into datetime format
ga_df['date']= pd.to_datetime(ga_df['date'],format='%m/%d/%y')

# Splitting the date column into year, month and day columns
ga_df['year']= ga_df['date'].dt.year
ga_df['month']= ga_df['date'].dt.month
ga_df['day']= ga_df['date'].dt.day

ga_df.head()

,date,users,year,month,day
0,2017-10-01,0,2017,10,1
1,2017-10-02,0,2017,10,2
2,2017-10-03,0,2017,10,3
3,2017-10-04,0,2017,10,4
4,2017-10-05,0,2017,10,5


In [4]:
# Saving Dataframe to CSV File
ga_df.to_csv("/home/chigoz/Downloads/Disso/00. Inputs/ga_data.csv", index=False, sep = ',')

## YouTube Data

To be used for Exploratory Analysis.

In [5]:
# Viewing video description dataframe
video_description_df.head()

,date,video_id,video_title,video_description
0,2021/06/22,3Cg8-eoNXdY,SLIM FIT DENIM JEANS LATEST PICK UPS + COLLECT...,This video has been sponsored by @FarFetch\nGE...
1,2021/06/13,zgXoCGUcR9w,"☀️☀️SUMMER OUTFIT PICKUPS UNBOXING | TMONCLER,...",THIS VIDEO HAS BEEN SPONSORED BY @SEVEN STORE\...
2,2021/06/09,AFpME2E2GAY,MONTHLY DESIGNER SELECTIONS UNBOXING & TRY-ON ...,THIS VIDEO HAS BEEN SPONSORED BY LUISAVIAROMA\...
3,2021/06/03,4TJCzY3Ph88,"ULTIMATE BALENCIAGA TRIPLE S , TRACK & SPEED S...",This video has been sponsored @Browns \n@Brown...
4,2021/05/27,TX3TFXp2y2M,🔥🔥 6 HOT MENSWEAR SUMMER SALES HAPPENING RIGHT...,https://thehoxtontrend.com/latest-spring-sales...


In [6]:
# Renaming Columns on Video Statistic Table
video_statistics_df.rename(columns={'views':'video_views', 'likes':'video_likes', 'dislikes':'video_dislikes', 'comments':'comments_count'}, inplace = True)
video_statistics_df.head()

,video_id,video_views,video_likes,video_dislikes,comments_count
0,3Cg8-eoNXdY,3275,106,8,36
1,zgXoCGUcR9w,4295,173,7,39
2,AFpME2E2GAY,5642,209,3,38
3,4TJCzY3Ph88,4134,136,6,38
4,TX3TFXp2y2M,7385,220,4,14


In [7]:
# Filling NAs in comment section with blanks
video_comments_df["comment"] = video_comments_df["comment"].fillna('-')

# Removing unneeded columns from dataframe
video_comments_df = video_comments_df[["video_id", "comment", "likes"]]

# Aggregating comments into list by video_id
video_comments = video_comments_df.groupby('video_id')['comment'].apply(list)
video_comments = pd.DataFrame(video_comments)
video_comments = video_comments.rename(columns={'comment':'video_comments'})

# Adding up comments likes by video_id
total_comments_likes = video_comments_df.groupby('video_id')['likes'].sum()
total_comments_likes = pd.DataFrame(total_comments_likes)
total_comments_likes = total_comments_likes.rename(columns={'likes':'total_comment_likes_count'})

# Merging all comments and total comments likes columns
aggregated_comments_df = video_comments.merge(total_comments_likes, on='video_id')
aggregated_comments_df.head()

,video_comments,total_comment_likes_count
video_id,,
-62No0ZxYdo,[CHECK OUT OUR LASTEST STONE ISLAND CHEST RIG ...,221
-_sRqyAV21g,[It will go either of two ways get more expens...,132
-gTkxpx8sok,"[Love it Hox. Keep making the great videos, lo...",241
0-ZCc21ot_M,[The pictures on the website didnt give these ...,212
0IQxwquZ1z8,[For steals and deals from Authorised Stockist...,137


In [8]:
# Filling NAs in tag_value section with blanks
video_tags_df["tag_value"] = video_tags_df["tag_value"].fillna('-')

# Aggregating tags into list by video_id
all_tags_df = video_tags_df.groupby('video_id')['tag_value'].apply(list)
all_tags_df = pd.DataFrame(all_tags_df)
all_tags_df = all_tags_df.rename(columns={'tag_value':'video_tags'})
all_tags_df.head()

,video_tags
video_id,
-62No0ZxYdo,"[harrods outlet westfield, harrods outlet, har..."
-_sRqyAV21g,"[mens fashion, tkmaxx outfit challenge, stone ..."
-gTkxpx8sok,"[mens fashion, tkmaxx outfit challenge, stone ..."
0-ZCc21ot_M,"[MA STRUM, MENS JACKETS, COMPETITION, AQUASCUT..."
0IQxwquZ1z8,"[mens fashion, canada goose, 2020 chilliwack, ..."


In [9]:
# Merging individual dataframes YouTube Data Dataframe for Exploratory Analysis
yt_df = video_description_df.merge(video_statistics_df, on='video_id')
yt_df = yt_df.merge(aggregated_comments_df, on='video_id')
yt_df = yt_df.merge(all_tags_df, on='video_id')

# Removing unneeded columns from dataframe
yt_df = yt_df[['date', 'video_id', 'video_title', 'video_description', 'video_tags', 'video_views', 'video_likes', 'video_dislikes', 'video_comments', 'comments_count', 'total_comment_likes_count']]

# Converting the date column into datetime format
yt_df['date']= pd.to_datetime(yt_df['date'],format='%Y/%m/%d')

# Splitting the date column into year, month and day columns
yt_df['year']= yt_df['date'].dt.year
yt_df['month']= yt_df['date'].dt.month
yt_df['day']= yt_df['date'].dt.day

yt_df.head()

,date,video_id,video_title,video_description,video_tags,video_views,video_likes,video_dislikes,video_comments,comments_count,total_comment_likes_count,year,month,day
0,2021-06-22,3Cg8-eoNXdY,SLIM FIT DENIM JEANS LATEST PICK UPS + COLLECT...,This video has been sponsored by @FarFetch\nGE...,"[mens fashion, Denim Jeans, Slim fit denim, Sk...",3275,106,8,[GET AN EXTRA 15% OFF SALE ITEMS @FARFETCH ...,36,31,2021,6,22
1,2021-06-13,zgXoCGUcR9w,"☀️☀️SUMMER OUTFIT PICKUPS UNBOXING | TMONCLER,...",THIS VIDEO HAS BEEN SPONSORED BY @SEVEN STORE\...,"[mens fashion, MENS SUMMER OUTFITS, MENS SUMME...",4295,173,7,[GET 15% OFF WITH CODE 'THT15' WORKS ON NEW SE...,39,21,2021,6,13
2,2021-06-09,AFpME2E2GAY,MONTHLY DESIGNER SELECTIONS UNBOXING & TRY-ON ...,THIS VIDEO HAS BEEN SPONSORED BY LUISAVIAROMA\...,"[mens fashion, tkmaxx outfit challenge, stone ...",5642,209,3,"[I like that orange moncler jacket!, My favour...",38,31,2021,6,9
3,2021-06-03,4TJCzY3Ph88,"ULTIMATE BALENCIAGA TRIPLE S , TRACK & SPEED S...",This video has been sponsored @Browns \n@Brown...,"[mens fashion, tkmaxx outfit challenge, stone ...",4134,136,6,[ENTER OUR 250 COMPETITION GIVEAWAY TO SPEND a...,38,34,2021,6,3
4,2021-05-27,TX3TFXp2y2M,🔥🔥 6 HOT MENSWEAR SUMMER SALES HAPPENING RIGHT...,https://thehoxtontrend.com/latest-spring-sales...,"[mens fashion, tkmaxx outfit challenge, stone ...",7385,220,4,"[Reppin the Patta collab! , Thanks picking up ...",14,14,2021,5,27


In [10]:
yt_df.to_csv("/home/chigoz/Downloads/Disso/00. Inputs/yt_data.csv", index=False, sep = ',')

## Numeric Data

To be used for Modelling.

In [11]:
# Selecting relevant columns from the google analytics dataframe
num_df = ga_df[['users', 'year', 'month', 'day']]

# Removing Dates where the website was not live
num_df = num_df.drop(list(range(0,37)))

# Selecting relevant columns from the youtube dataframe for the youtube (numeric data) dataframe
yt_num = yt_df[['video_id', 'year', 'month', 'day', 'video_views', 'video_likes', 'video_dislikes', 'comments_count', 'total_comment_likes_count']] 

# Merging the google analytics and youtube (numeric data) dataframe to create the numeric data dataframe
num_df = num_df.merge(yt_num, on=['year', 'month', 'day'], how = 'left')

# Forward filling empty rows in numeric dataframe
# These are days when no new video is released so the last video's statistics are used
num_df = num_df.replace(0, np.nan).ffill()

num_df.head()

,users,year,month,day,video_id,video_views,video_likes,video_dislikes,comments_count,total_comment_likes_count
0,40,2017,11,7,3r3lWi_T45g,11564.0,345.0,2.0,67.0,52.0
1,90,2017,11,8,3r3lWi_T45g,11564.0,345.0,2.0,67.0,52.0
2,79,2017,11,9,3r3lWi_T45g,11564.0,345.0,2.0,67.0,52.0
3,16,2017,11,10,3r3lWi_T45g,11564.0,345.0,2.0,67.0,52.0
4,10,2017,11,11,3r3lWi_T45g,11564.0,345.0,2.0,67.0,52.0


In [12]:
# Saving Dataframe to CSV File
num_df.to_csv("/home/chigoz/Downloads/Disso/00. Inputs/num_data.csv", index=False, sep = ',')

## Textual Data

To be used for Modelling.

In [13]:
# Selecting relevant columns for the dataframe
text_df = yt_df[['video_id','video_title', 'video_description', 'video_tags', 'video_comments']]

text_df.head()

,video_id,video_title,video_description,video_tags,video_comments
0,3Cg8-eoNXdY,SLIM FIT DENIM JEANS LATEST PICK UPS + COLLECT...,This video has been sponsored by @FarFetch\nGE...,"[mens fashion, Denim Jeans, Slim fit denim, Sk...",[GET AN EXTRA 15% OFF SALE ITEMS @FARFETCH ...
1,zgXoCGUcR9w,"☀️☀️SUMMER OUTFIT PICKUPS UNBOXING | TMONCLER,...",THIS VIDEO HAS BEEN SPONSORED BY @SEVEN STORE\...,"[mens fashion, MENS SUMMER OUTFITS, MENS SUMME...",[GET 15% OFF WITH CODE 'THT15' WORKS ON NEW SE...
2,AFpME2E2GAY,MONTHLY DESIGNER SELECTIONS UNBOXING & TRY-ON ...,THIS VIDEO HAS BEEN SPONSORED BY LUISAVIAROMA\...,"[mens fashion, tkmaxx outfit challenge, stone ...","[I like that orange moncler jacket!, My favour..."
3,4TJCzY3Ph88,"ULTIMATE BALENCIAGA TRIPLE S , TRACK & SPEED S...",This video has been sponsored @Browns \n@Brown...,"[mens fashion, tkmaxx outfit challenge, stone ...",[ENTER OUR 250 COMPETITION GIVEAWAY TO SPEND a...
4,TX3TFXp2y2M,🔥🔥 6 HOT MENSWEAR SUMMER SALES HAPPENING RIGHT...,https://thehoxtontrend.com/latest-spring-sales...,"[mens fashion, tkmaxx outfit challenge, stone ...","[Reppin the Patta collab! , Thanks picking up ..."


In [14]:
# Saving Dataframe to CSV File
text_df.to_csv("/home/chigoz/Downloads/Disso/00. Inputs/text_data.csv", index=False, sep = ',')